# This notebook samples a random set of devices by cuebiq Id from the set of all pings that were observed in the Gulf polygon

## Key Inputs: 
* Spatial wkts with Gulf and island areas to be excluded
* Dedicated table with pings observed within the Gulf. 

## Key Output:
* **Pings_OurTable_Gulf_ALL.csv** -- a csv file with all pings of randomly selected devices within the Gulf

In [ ]:
!pip install pandarallel

In [ ]:
# Install modules
!pip install tqdm

# # Suppress all warnings
# import warnings
# warnings.filterwarnings("ignore")

import pandas as pd
import os
import sys
import time
import csv
from datetime import datetime, timedelta
from datetime import datetime
import pytz  
import warnings
import numpy as np
import math
local_timezone = pytz.timezone('US/Central')

import csv
import datetime
import datetime as dt
import geopandas as gpd
import heapq
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pytz
import requests
import seaborn as sns
import time
import math

from scipy import interpolate
from shapely import wkt
from shapely.geometry import Polygon
from zipfile import ZipFile
from shapely.geometry import Point  # Import the Point class from shapely.geometry
from datetime import datetime
from tqdm import tqdm

class OperationCancelled(Exception):
    pass

local_timezone = pytz.timezone('US/Central')

### Query code

In [ ]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [ ]:
%sql trino://localhost:9090/cuebiq/

In [ ]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [ ]:
def get_dates_sequence(
    date_start, 
    date_end, 
    date_format
):
    return [
        (datetime.strptime(date_start, date_format) + timedelta(days=x)).strftime(date_format)
        for x in range (
        0,
        (datetime.strptime(date_end, date_format) - datetime.strptime(date_start, date_format) + timedelta(days=1)).days
        )
    ]

### Date parameter for request from Spectus

In [ ]:
date_format = "%Y%m%d"

first_date = "20190101"
last_date_to_compute = "20220422"

In [ ]:
dates_to_compute = get_dates_sequence(first_date, last_date_to_compute, date_format)

# Functions and other objects

### Gulf wkt 

In [ ]:
Gulf= 'POLYGON ((-97.1461459379788 25.957168007570218, -96.289339604698 24.85679572942162, -92.46567751398405 24.93226258740249, -86.46809930536585 26.63685607835417, -87.4518747436496 30.2870218479717, -87.56970870420537 30.265986917324028, -87.68688306481553 30.240398581969316, -87.80866540589953 30.223904790285445, -87.86330288865585 30.223335989499247, -87.95809562982397 30.227886303681714, -88.02919018569986 30.21821663439117, -88.04103927834595 30.236417575256624, -87.96007047859824 30.256320996255198, -87.9264980494346 30.259732606445226, -87.87712683007626 30.243810745100816, -87.84223783506322 30.25688960618315, -87.78825863523141 30.25802681616385, -87.76653529871386 30.268829654214812, -87.75929418654121 30.287589653053473, -87.77443469381102 30.308051015443525, -87.80142429372691 30.321121316921534, -87.83236359119138 30.34441622544655, -87.84487096676237 30.372249210920785, -87.87910145959437 30.384175375165654, -87.9185984350809 30.414267849534085, -87.92386469847926 30.462509996885288, -87.94032177159842 30.47215556160502, -87.93505550820048 30.49938493987453, -87.91991500093027 30.529441859299524, -87.9172818692315 30.552120171744733, -87.91991500093027 30.575926698627413, -87.92452298140375 30.609925886488114, -87.92978924480211 30.64051495391115, -87.9475628837711 30.650709160976575, -87.96928622028864 30.651841784313817, -87.99298440558087 30.665432229337696, -88.01141632747473 30.659203512573512, -88.0423556249392 30.638249428634225, -88.05091330296123 30.624088693607646, -88.04696360541267 30.61559125795084, -88.05354643466042 30.590094476474817, -88.07790290287716 30.565157802896906, -88.07592805410286 30.539080789195197, -88.09370063563347 30.456267467088836, -88.09896689903184 30.398370320370773, -88.10620801120405 30.353504810736837, -88.15887064518647 30.323393627447288, -88.19178479142525 30.31032362908067, -88.20429216699586 30.330780246575657, -88.19705105482359 30.353504810736837, -88.28855238136764 30.381335210261483, -88.2957934935403 30.364865114151982, -88.31817511298274 30.363161152750493, -88.32014996175704 30.38190309511809, -88.3517475421464 30.390988804016374, -88.39058623470804 30.37224860380745, -88.39256108348229 30.342143188113525, -88.43732432236718 30.34725608152752, -88.44259058576552 30.331348424972347, -88.48077099540268 30.308618718161995, -88.5018360489957 30.312596797505776, -88.5288256489116 30.337030027619335, -88.55384040005282 30.331348424972347, -88.58807111214138 30.335893733464147, -88.60584475111035 30.368840908630105, -88.65784982716818 30.350096668434418, -88.70377316315728 30.333940318201655, -88.76345157602148 30.340304903230376, -88.81215810808295 30.366323547786152, -88.96059706293589 30.3823315816322, -89.10671665911954 30.334299618169936, -89.33169445006897 30.2782325348214, -89.41287200350473 30.192067121425936, -89.44534302487904 30.08977311890679, -89.359526754104 30.069703042406346, -89.20876844058175 30.176028017449696, -89.16006190852028 30.16199165952355, -89.16006190852028 30.08977311890679, -89.13918768049385 30.055651566395866, -89.16238126718959 29.921058052351142, -89.21108779925103 29.7782302647352, -89.24587817929464 29.66946405486304, -89.31545893938255 29.693644509171918, -89.27371048332964 29.629150377050877, -89.30386214603394 29.59688781400496, -89.3850396994697 29.61503677626405, -89.46157853556547 29.532331728567527, -89.3827203407997 29.439460132099583, -89.18979122539224 29.427128425449155, -89.19596940111164 29.394836842780364, -88.95749181835598 29.20950068771296, -89.04027937299146 29.04327521681688, -89.10329676532581 29.036793464759455, -89.13913018449634 28.94816874403513, -89.19473376596763 28.98276303815456, -89.26022242858967 29.027070073382717, -89.29358457747252 29.016265231135534, -89.4010848349841 28.903827915763785, -89.44186079473003 28.94816874403513, -89.39243538897739 29.025989640029877, -89.44680333530505 29.0864764893534, -89.45298151102406 29.14692783767063, -89.42209063242889 29.17066685855066, -89.376372132108 29.156639918576943, -89.36525141581359 29.180376694261838, -89.34671688865649 29.20195081707395, -89.37390086182032 29.206265096965808, -89.40232047012778 29.203029404070804, -89.4356826190106 29.20195081707395, -89.46780913274982 29.179297868997324, -89.49499310591362 29.193321712569713, -89.54441851166625 29.2278337721458, -89.63214860687673 29.254788228681946, -89.68404528291667 29.282813330704627, -89.80143062157865 29.305443379957012, -89.85579856790665 29.300055727855437, -89.9076952439466 29.27526886673428, -89.98924716343802 29.21489311194422, -90.05473582606008 29.165272108427317, -90.1572935429963 29.102672297767754, -90.26232567572231 29.06703684192044, -90.2833314731671 29.10914859304988, -90.26479694600961 29.13181701694839, -90.28827401374208 29.15232223739727, -90.29939473003648 29.19547774319095, -90.269739486585 29.23538042243976, -90.29568782460515 29.2310673685935, -90.30927981118687 29.24724038352842, -90.33893505463838 29.26341084252479, -90.393303000966 29.251552755878066, -90.40565935240406 29.30220951059583, -90.44767094729364 29.334530475168393, -90.50203889362164 29.32698983267116, -90.54899302908619 29.3032870410304, -90.57123446167503 29.309751984787326, -90.56629192109963 29.291433580776584, -90.59100462397578 29.241849662499888, -90.72198194921984 29.166349767170132, -90.74669465209598 29.108069019686837, -90.77634989554745 29.09943202507891, -90.84801673388853 29.115625795370804, -90.88755705849043 29.13721352427197, -90.89126396392177 29.16742872854931, -90.91474103165422 29.217048688862008, -90.92956865337999 29.198713674444775, -90.93080428852403 29.16742872854931, -90.97652278884493 29.160954790180483, -91.02965510002853 29.163112814994165, -91.08772995178788 29.17713887061521, -91.15445424955357 29.210577881865433, -91.21005783102524 29.21165637807694, -91.28790284508534 29.240771484217632, -91.33609261569393 29.292511224930067, -91.36327658885774 29.33237606278429, -91.36698404442399 29.461560546417502, -91.37316222014341 29.483075264842185, -91.42505889618336 29.473394206552683, -91.48189811279867 29.486302078781378, -91.50784645081843 29.51963980456361, -91.52885224826323 29.514263494469418, -91.5424442348453 29.538992155608895, -91.5424442348453 29.565864276975645, -91.56097876200242 29.62710598028727, -91.58692710002259 29.62603188493061, -91.6351168706312 29.61636451146687, -91.65118012750057 29.642142112800144, -91.63758814091887 29.656102225576717, -91.64129504635021 29.723727639327834, -91.71296188469128 29.718362205888425, -91.76980110130658 29.702264184791886, -91.83034722335326 29.685090118027063, -91.85505992622939 29.662544701481323, -91.845174845079 29.634624327137942, -91.79451380418274 29.609919080630874, -91.75744474986853 29.590580315799386, -91.69689862782151 29.57123784364269, -91.71914006041031 29.53254178333445, -91.74879530386181 29.517489314800784, -91.77103673645024 29.48199963736188, -91.8019276150454 29.470166981654714, -91.8525886559417 29.47662132872088, -91.94649692687157 29.517489314800784, -92.01816376521265 29.556191136499905, -92.081181157547 29.575536491098305, -92.14666982016863 29.57338719024669, -92.2282217396605 29.543292175193457, -92.2620745477254 29.513938983582918, -92.40184398133182 29.53587031357371, -92.53015690398729 29.55580375974114, -92.65388722226176 29.58171136156176, -92.76845233177524 29.613588513778055, -92.90134785881101 29.66735835342932, -92.99758255080252 29.705179169035475, -93.10985635812567 29.739006777997886, -93.2015084457365 29.758900043967557, -93.27024751144474 29.76088915343348, -93.3389865771526 29.740996282222838, -93.43980387352478 29.750943211286994, -93.61623414217547 29.740996282222838, -93.76058618016258 29.715129649330578, -93.8293252458708 29.663376387168597, -93.95534686633557 29.6852752506843, -94.10199020651301 29.65541198179467, -94.53962892485461 29.488014011072636, -94.71376789131534 29.402214927729204, -94.72293005770678 29.324334651639106, -94.9360211614021 29.206402439154715, -95.13765575414574 29.060298314788838, -95.37824248412433 28.86583930097359, -95.42635983011992 28.85781253647636, -95.88003908572986 28.63281185841163, -96.17332576608472 28.508049911739334, -96.3657951500674 28.385156078353774, -96.42995161139497 28.326681104162276, -96.5811775559527 28.237899002445175, -96.8538425165949 28.05404703841444, -97.01423366991399 27.871905659410203, -97.18378864850824 27.632633899635536, -97.28460594488008 27.443685488256804, -97.34647110401767 27.286998971872194, -97.37396673030096 27.111736817761667, -97.37167542811062 26.942327753778045, -97.30062980709232 26.653886203530476, -97.27907391837897 26.589650805967054, -97.227579295341 26.438555761793396, -97.19764056101677 26.3183955849306, -97.16770182669217 26.152971688301193, -97.15093613547081 26.036819954392655, -97.1461459379788 25.957168007570218))'
Gulf_with_coast_Coarse = 'POLYGON ((-84.84686264156812 25.68340767152216, -87.43248750342957 30.94498325396067, -95.80214228446721 30.0359530662285, -98.3915463041922 26.769804868308682, -95.4673270575147 24.286120603664912, -84.84686264156812 25.68340767152216))'

### Islands with lots of pings on it that should be excluded from feature calculation

In [ ]:

AlabamaIsland= 'POLYGON ((-88.11218406491841 30.28270197801872, -88.11748790022997 30.284420403489747, -88.1289123735651 30.296676657854306, -88.13525930319557 30.31441063898764, -88.12276411652998 30.344306892409193, -88.1261384293325 30.346573465262267, -88.14068302824425 30.330248934489845, -88.14841474252134 30.326463909723657, -88.14033683208271 30.309429488292594, -88.12233483298868 30.27266101723977, -88.13929844491092 30.259205656184136, -88.17437965596034 30.254122039939332, -88.2182311697725 30.24923753336725, -88.30108761392364 30.233386081193586, -88.32532134524112 30.23258845903686, -88.34528532389731 30.2314917180055, -88.40194641363063 30.215039068387483, -88.40817794454074 30.217232905188055, -88.42087180380169 30.212147117858834, -88.43252707457806 30.211449048099738, -88.44764430697126 30.20376995391028, -88.4625307419233 30.204368346411314, -88.48353330906463 30.209653988836138, -88.51157452826871 30.22032416074346, -88.51088213594514 30.2246118384801, -88.53211550052762 30.228699915260023, -88.57377443864924 30.22780254711091, -88.68005685360352 30.245947643114775, -88.7160612544177 30.256215137818614, -88.73648716752392 30.24843987369816, -88.7441034830807 30.249536425624242, -88.76568304382471 30.244851072739735, -88.77803070692467 30.247742061555087, -88.78576242120177 30.24166091740736, -88.86804184612036 30.248938361474544, -88.8717346051779 30.25133081048743, -88.89908410195018 30.234083937046506, -88.91223955609405 30.225409510211975, -88.9413202941934 30.222318343593045, -88.98551800416709 30.215836746861996, -88.9950960979732 30.208656633512476, -89.06366907456174 30.218130299450735, -89.05593736028412 30.237573354839526, -89.06678484001655 30.251729468053952, -89.08051728776313 30.235080870280683, -89.13729363724818 30.23378483291677, -89.1603733813597 30.22989637955409, -89.12047958599241 30.22005260021551, -89.12145473065867 30.211288948999467, -89.09844131653294 30.20977208396677, -89.10448721346415 30.20286162516733, -89.08849484093591 30.203030178716162, -89.0746477866739 30.219378500914573, -88.99020129980069 30.207581121374503, -88.97615921660534 30.20437870267233, -88.95626626541215 30.205221454538503, -88.93988353780959 30.21752474053794, -88.91335960288517 30.224434169581016, -88.90282804048856 30.221737864773814, -88.89112630449237 30.229152525120753, -88.8747438740979 30.238588556974634, -88.86908803503293 30.248529110671157, -88.78542036054029 30.240947271347025, -88.77430371134409 30.238082926154163, -88.75870139668251 30.23572399096632, -88.732957577491 30.23825141935825, -88.69941263547238 30.232690225631984, -88.64343933162394 30.22308510875915, -88.59780256123877 30.21954614500025, -88.57654440751222 30.227466506513878, -88.53129735506896 30.228477021928455, -88.52973712360316 30.21988264595238, -88.52349619773817 30.217523274113134, -88.51160410515517 30.21952563990618, -88.50750849755646 30.207053988309582, -88.46518721903666 30.192389349352695, -88.41935541971861 30.19660354973145, -88.40141275785743 30.214301221513225, -88.34641459868976 30.230142150145852, -88.30409253525568 30.224580980238727, -88.22452073048152 30.239072330381035, -88.17205794743222 30.243958466852533, -88.12642117704708 30.227108695087196, -88.07200810466486 30.2368819144896, -88.07493353866359 30.258783876570888, -88.11218406491841 30.28270197801872))'

GalvestonIsland = 'POLYGON ((-94.79810501429436 29.285782165982226, -94.85587801828437 29.256474439597326, -94.96386123262589 29.194262486622222, -94.98840851114379 29.17802706499387, -94.9832015126703 29.172398185737194, -94.95751050670931 29.185942592884174, -94.93155434112559 29.200608310191058, -94.90207396798999 29.21719558256349, -94.87503715080425 29.23384831840025, -94.84904021158258 29.24836617943305, -94.82392719885846 29.263585041171126, -94.80006095983339 29.275328868980722, -94.79198685607093 29.280744467360933, -94.78065199926338 29.29067598019759, -94.7849675640115 29.294349034860943, -94.79810501429436 29.285782165982226))'

BiscayneTX = 'POLYGON ((-94.72630885515686 29.39417826702656, -94.71635868628177 29.38842018852729, -94.70841340220476 29.39890094161126, -94.69809195840361 29.409315928554435, -94.6909634792623 29.41675455187773, -94.6995770582621 29.419341771414878, -94.72630885515686 29.39417826702656))'

# Function to extract coordinates from polygon string
def extract_coordinates(polygon):
    coordinates = polygon.split('((')[1].split('))')[0].split(', ')
    return [(float(coord.split()[0]), float(coord.split()[1])) for coord in coordinates]

# Extract coordinates for each polygon
alabama_coordinates = extract_coordinates(AlabamaIsland)
galveston_coordinates = extract_coordinates(GalvestonIsland)
biscayne_coordinates = extract_coordinates(BiscayneTX)

# Find the largest and smallest latitudes and longitudes
all_coordinates = alabama_coordinates + galveston_coordinates + biscayne_coordinates

min_islands_latitude = min(coord[1] for coord in all_coordinates)
max_islands_latitude = max(coord[1] for coord in all_coordinates)
min_islands_longitude = min(coord[0] for coord in all_coordinates)
max_islands_longitude = max(coord[0] for coord in all_coordinates)


### Function that excludes rows that are within the islands

In [ ]:
from shapely import wkt

polygon_wkts = [AlabamaIsland, GalvestonIsland, BiscayneTX]

def filter_points_outside_polygons(df, polygon_wkts):
    """
    Filter rows from a DataFrame where lat and lng are outside the specified polygons.

    Parameters:
    df (DataFrame): The DataFrame containing lat and lng columns.
    polygon_wkts (list): A list of WKT representations of polygons.

    Returns:
    DataFrame: The filtered DataFrame.
    """
    # Parse the WKT representations of the polygons to create Shapely geometry objects
    polygons = [wkt.loads(polygon_wkt) for polygon_wkt in polygon_wkts]

    # Create a boolean mask to identify points outside any of the polygons
    mask = df.apply(lambda row: all(not polygon.contains(Point(row['lng'], row['lat'])) for polygon in polygons), axis=1)

    # Filter rows outside any of the polygons
    filtered_df = df[mask]

    return filtered_df


In [ ]:
from shapely.geometry import Polygon, Point  # Import the Point class

# AlabamaIsland reformatting
coordinates_str = AlabamaIsland.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
alabama_island_polygon = Polygon(coordinates_list)

# GalvestonIsland reformatting
coordinates_str = GalvestonIsland.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
galveston_island_polygon = Polygon(coordinates_list)

# BiscayneTX reformatting
coordinates_str = BiscayneTX.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
biscayne_tx_polygon = Polygon(coordinates_list)

# Create a list of polygons
island_polygons = [alabama_island_polygon, galveston_island_polygon, biscayne_tx_polygon]

## , min_islands_latitude, max_islands_latitude, min_islands_longitude, max_islands_longitude
def is_point_outside_all_islands(data_frame):
    # Filter out points not in the Gulf but outside the islands rectangle
    data_frame['in_Gulf_not_Islands_rectangle'] = (data_frame['lat'] < min_islands_latitude) | \
                                                   (data_frame['lat'] > max_islands_latitude) | \
                                                   (data_frame['lng'] < min_islands_longitude) | \
                                                   (data_frame['lng'] > max_islands_longitude)
    filtered_data = data_frame[data_frame['in_Gulf_not_Islands_rectangle'] == False]
    
    # Apply the function is_point_outside_all_islands to the filtered data
    filtered_data['in_Gulf_not_Islands'] = filtered_data.apply(is_point_outside_all_islands_row, axis=1)
    
    return filtered_data

def is_point_outside_all_islands_row(row):
    lat = row['lat']
    lng = row['lng']
    point = Point(lng, lat)
    return all(not polygon.contains(point) for polygon in island_polygons)



### Function that finds out if a point is inside the Gulf wkt and not in the islands

In [ ]:
from shapely.geometry import Polygon, Point  

# AlabamaIsland reformatting
coordinates_str = AlabamaIsland.replace('POLYGON ((', '').replace('))', '')  
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  
alabama_island_polygon = Polygon(coordinates_list)

# GalvestonIsland reformatting
coordinates_str = GalvestonIsland.replace('POLYGON ((', '').replace('))', '')  
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  
galveston_island_polygon = Polygon(coordinates_list)

# BiscayneTX reformatting
coordinates_str = BiscayneTX.replace('POLYGON ((', '').replace('))', '')  
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  
biscayne_tx_polygon = Polygon(coordinates_list)

# Create a list of polygons
island_polygons = [alabama_island_polygon, galveston_island_polygon, biscayne_tx_polygon]

# Gulf reformatting
coordinates_str = Gulf.replace('POLYGON ((', '').replace('))', '')  
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  
gulf_polygon = Polygon(coordinates_list)

# Gulf with coast
coordinates_str = Gulf_with_coast_Coarse.replace('POLYGON ((', '').replace('))', '')  
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  
gulf_with_coast_polygon = Polygon(coordinates_list)

# Create a function to check if a point is inside the Gulf and not inside any island polygon
# def from shapely.geometry import Polygon, Point  # Import the Point class

# AlabamaIsland reformatting
coordinates_str = AlabamaIsland.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
alabama_island_polygon = Polygon(coordinates_list)

# GalvestonIsland reformatting
coordinates_str = GalvestonIsland.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
galveston_island_polygon = Polygon(coordinates_list)

# BiscayneTX reformatting
coordinates_str = BiscayneTX.replace('POLYGON ((', '').replace('))', '')  # Remove the 'POLYGON ((' and '))'
coordinates_list = [tuple(map(float, point.split())) for point in coordinates_str.split(',')]  # Split and convert to tuples
# Create a Polygon object using the list of coordinates
biscayne_tx_polygon = Polygon(coordinates_list)

# Create a list of polygons
island_polygons = [alabama_island_polygon, galveston_island_polygon, biscayne_tx_polygon]

# Create a function to check if a point is outside all polygons
def is_point_in_Gulf_not_Islands(row):
    lat = row['lat']
    lng = row['lng']
    point = Point(lng, lat)
    
    # Check if the point is inside the Gulf polygon
    if gulf_polygon.contains(point):
        # Check if the point is not inside any island polygon
        if all(not polygon.contains(point) for polygon in island_polygons):
            return True
    
    return False

def is_point_in_Gulf_Coast_or_Islands(row):
    lat = row['lat']
    lng = row['lng']
    point = Point(lng, lat)
    
    # Check if the point is inside the Gulf polygon
    if gulf_with_coast_polygon.contains(point):
        return True
    return False
    
    

In [ ]:
import os

def check_file_existence(file_path):
    if os.path.exists(file_path):
        print(f"Does     Exist {file_path}")
    else:
        print(f"Does NOT Exist {file_path}")



# Set directories to be used

In [ ]:
#################################################################
# Directory for this output
OurTable_V3_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files'
# Expand the tilde to the user's home directory
OurTable_V3_directory = os.path.expanduser(OurTable_V3_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(OurTable_V3_directory)
print(OurTable_V3_directory, "exists = " ,DirExist)

#################################################################
# Directory some core data fro analysis
CoreData_Directory = '~/RecFishing/CoreData'
CoreData_Directory = os.path.expanduser(CoreData_Directory)
print(CoreData_Directory, "exists = ", os.path.exists(CoreData_Directory))

#################################################################
# Directory  with Original_directory material
Original_directory = '~/RecFishing/DataflowStudioJobs'
Original_directory = os.path.expanduser(Original_directory)
DirExist = os.path.exists(Original_directory)
print(Original_directory, "exists = ", DirExist)


#################################################################
# Directory  with Original_directory material
Previously_Processed_directory = '~/RecFishing/DataflowStudioJobs/FinalCode - Rec Fishing Identification'
Previously_Processed_directory = os.path.expanduser(Previously_Processed_directory)
DirExist = os.path.exists(Previously_Processed_directory)
print(Previously_Processed_directory, "exists = ", DirExist)

#################################################################
# Directory with Travel Cost files
Travel_Cost_directory = '~/RecFishing/Travel Costs with Dedicated Table/CSV Files'
# Expand the tilde to the user's home directory
Travel_Cost_directory = os.path.expanduser(Travel_Cost_directory)
DirExist = os.path.exists(Travel_Cost_directory)
print(Travel_Cost_directory, "exists = ", DirExist)

#################################################################
# Directory with Weather data and related files
Weather_Data_directory = '~/RecFishing/uploaded_files/Weather Data'
# Expand the tilde to the user's home directory
Weather_Data_directory = os.path.expanduser(Weather_Data_directory)
print(Weather_Data_directory, "exists = ", DirExist)


# ID_list_RandomSample from ScheduledExecution5.pkl


# Set input and output files to be used

In [ ]:
# Complete list of randomized IDs
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list.pkl')
# check_file_existence(PKL_File_With_Random_IDs_Filename)

# Complete list of randomized IDs- without bernouli sampling 740k
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
PKL_File_With_Random_IDs_Filename  =  os.path.join(CoreData_Directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
check_file_existence(PKL_File_With_Random_IDs_Filename)
    
# Data gathered and used prior to the NOAA Webinar in February 2024
IDs_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'IDs_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(IDs_Used_in_NOAA_Webinar_filename)
Ping_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'Pings_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(Ping_Used_in_NOAA_Webinar_filename)
                                                     
# List of IDs that have been processed for Indicators
AlreadyFullyProcessedIDs_Filename  =  os.path.join(OurTable_V3_directory, 'RandomlyChosenCuebiq_ids.List_of_Processed_ids.csv')
check_file_existence(AlreadyFullyProcessedIDs_Filename)
    
# List of IDs that have been Pulled from our  dedicated table
IDs_Pulled_from_Dedicated_Table_filename  =  os.path.join(OurTable_V3_directory, 'IDs_Pulled_From_Dedicated_Table.csv')
check_file_existence(IDs_Pulled_from_Dedicated_Table_filename)
    

# Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_OurTable_temp_filename = os.path.join(OurTable_V3_directory,'Pings_OurTable_temp.csv')
check_file_existence(Pings_OurTable_temp_filename)
    
# Pings from V3 corresponding with the IDs found in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_V3_temp_filename = os.path.join(OurTable_V3_directory,'Pings_V3_temp.csv')
check_file_existence(Pings_V3_temp_filename)
    
# Set output file names
Indicators_IDs_checked_filename = os.path.join(OurTable_V3_directory, 'IDs_Checked_Indicators_OurTable.csv')
check_file_existence(Indicators_IDs_checked_filename)

# List of IDs and dates for V3 query Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
OurTable_IDs_and_Dates_filename = os.path.join(OurTable_V3_directory,'OurTable_IDs_and_Dates.csv')
check_file_existence(OurTable_IDs_and_Dates_filename)
    
# Filenames for consolidated data frames
Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Pings_OurTable_Gulf_filename)

Pings_OurTable_Gulf_MT19_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_MT19.csv')
check_file_existence(Pings_OurTable_Gulf_MT19_filename)

Pings_OurTable_Coast_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Coast.csv')
check_file_existence(Pings_OurTable_Coast_filename)

Pings_OurTable_Outside_our_wkts_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Outside_our_wkts.csv')
check_file_existence(Pings_OurTable_Outside_our_wkts_filename)
    
    
### Dedicate Table Names for reference
# Dedicated table with all Pings within the Gulf WKT for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5.DeviceTable   
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-updated.ipynb

# Dedicated table with all Pings within the Gulf WKT AND Origin for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5_parallel_origin.DeviceTable
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-origin.ipynb


### Function that creates the OurTable_IDS_and_Dates files that will be used for efficient queries Spectus device table

In [ ]:
import pandas as pd
import os

def create_OurTable_IDS_and_Dates(df):
    eight_hours = 8 * 60 * 60

    # Create date variable for event
    df['event_date'] = pd.to_datetime(df['event_timestamp'], unit='s')
    df['event_date_int'] = df['event_date'].dt.strftime('%Y%m%d').astype(int)

    # Create date variable for 8 hours before
    df['event_timestamp_8hrs_before'] = df['event_timestamp'] - eight_hours
    df['event_date_8hrs_before'] = pd.to_datetime(df['event_timestamp_8hrs_before'], unit='s')
    df['event_date_int_8hrs_before'] = df['event_date_8hrs_before'].dt.strftime('%Y%m%d').astype(int)

    # Create date variable for 8 hours after
    df['event_timestamp_8hrs_after'] = df['event_timestamp'] + eight_hours
    df['event_date_8hrs_after'] = pd.to_datetime(df['event_timestamp_8hrs_after'], unit='s')
    df['event_date_int_8hrs_after'] = df['event_date_8hrs_after'].dt.strftime('%Y%m%d').astype(int)

    # Drop duplicates based on specific columns
    df.drop_duplicates(subset=['cuebiq_id', 'event_date_int_8hrs_after', 'event_date_int_8hrs_before'], inplace=True)

    # Sort values by event timestamp
    df.sort_values(by='event_timestamp', inplace=True)

    # Drop rows where 'event_date_int' is less than 20190101
    df = df[df['event_date_int'] >= 20190101]

    # Keep only necessary columns
    df = df[['cuebiq_id', 'event_date_int', 'event_date_int_8hrs_before', 'event_date_int_8hrs_after']]

    return df



# Full Loop for gathering new pings

## Step 0 - Load the PKL list of all IDs and remove IDs that have already been fully processed using ML pulled from cuebiq_id_list.pkl

In [ ]:
import pickle
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list.pkl')
# cuebiq_id_list_wo_sampling

# Load the list from the file
with open(PKL_File_With_Random_IDs_Filename, 'rb') as file:
    cuebiq_id_list = pickle.load(file)
# cuebiq_id_list

print("The complete list of ids from which we are sampling has", len(cuebiq_id_list), "devices.")

In [ ]:
# Exclude IDs that were drawn and used prior to the NOAA Webinar
def Load_ID_list_and_drop_old_IDs(IDs_Used_in_NOAA_Webinar_filename):
    # Load the list from the file
    with open(PKL_File_With_Random_IDs_Filename, 'rb') as file:
        cuebiq_id_list = pickle.load(file)

    print("PKL loaded", len(cuebiq_id_list))
    if os.path.exists(IDs_Used_in_NOAA_Webinar_filename):
        AlreadyFullyProcessedIDs_df = pd.read_csv(IDs_Used_in_NOAA_Webinar_filename)
        processed_ids_set = set(AlreadyFullyProcessedIDs_df['cuebiq_id'])
        cuebiq_id_list_remaining = [id for id in cuebiq_id_list if id not in processed_ids_set]

    print("There are", len(AlreadyFullyProcessedIDs_df), "that were gathered prior to the NOAA Webinar, so there are", len(cuebiq_id_list_remaining),"of", len(cuebiq_id_list), "yet to be processed")
    cuebiq_id_list=cuebiq_id_list_remaining
    
    return cuebiq_id_list 

In [ ]:
newlist = Load_ID_list_and_drop_old_IDs(IDs_Used_in_NOAA_Webinar_filename)
AlreadyFullyProcessedIDs_df.head()

In [ ]:
if os.path.exists(IDs_Pulled_from_Dedicated_Table_filename):
    AlreadyFullyProcessedIDs_df = pd.read_csv(IDs_Pulled_from_Dedicated_Table_filename)
    processed_ids_set = set(AlreadyFullyProcessedIDs_df['cuebiq_id'])
    cuebiq_id_list_remaining = [id for id in cuebiq_id_list if id not in processed_ids_set]
    print("There are", len(AlreadyFullyProcessedIDs_df), "that have been processed from an initial total of ", len(cuebiq_id_list),"and", len(cuebiq_id_list_remaining), "yet to be processed")
    cuebiq_id_list=cuebiq_id_list_remaining
else:
    print(IDs_Pulled_from_Dedicated_Table_filename,"does not exist")

In [ ]:
def remove_recently_pulled_ID(IDs_Pulled_from_Dedicated_Table_filename, cuebiq_id_list):
    with open(PKL_File_With_Random_IDs_Filename, 'rb') as file:
        cuebiq_id_list = pickle.load(file)

    if os.path.exists(IDs_Pulled_from_Dedicated_Table_filename):
        AlreadyFullyProcessedIDs_df = pd.read_csv(IDs_Pulled_from_Dedicated_Table_filename)
        nTotal = len(AlreadyFullyProcessedIDs_df)  # Update nTotal
        
        processed_ids_set = set(AlreadyFullyProcessedIDs_df['cuebiq_id'])
        cuebiq_id_list_remaining = [id for id in cuebiq_id_list if id not in processed_ids_set]
        print("There are", len(AlreadyFullyProcessedIDs_df), "that have been processed from an initial total of ", len(cuebiq_id_list),"and", len(cuebiq_id_list_remaining), "yet to be processed")
        cuebiq_id_list = cuebiq_id_list_remaining
    else:
        nTotal = 0
        print(IDs_Pulled_from_Dedicated_Table_filename, "does not exist")
    
    return cuebiq_id_list, nTotal


## Step 1: Pull Pings from OurTable inside Gulf
a) Grab large set of ids from 
Save this as Pings_OurTable_temp.csv

##### Load the pickle with 740k pings

In [ ]:
def LoadPkl():
    with open(PKL_File_With_Random_IDs_Filename, 'rb') as file:
        cuebiq_id_list = pickle.load(file)
    print("There are", len(cuebiq_id_list)," IDs in the complete list of randomly drawn IDs")
    return cuebiq_id_list

##### Drop from the list IDs that have already been pulled  as stored in IDs_Pulled_from_Dedicated_Table_filename

In [ ]:
def EliminatePreviouslyFoundIDs(cuebiq_id_list):
    nTotalPrevious=0
    if os.path.exists(IDs_Pulled_from_Dedicated_Table_filename):
        AlreadyFullyProcessedIDs_df = pd.read_csv(IDs_Pulled_from_Dedicated_Table_filename)
        nTotalPrevious = len(AlreadyFullyProcessedIDs_df)  # Update nTotal

        processed_ids_set = set(AlreadyFullyProcessedIDs_df['cuebiq_id'])
        cuebiq_id_list_remaining = [id for id in cuebiq_id_list if id not in processed_ids_set]
        print("There are", nTotalPrevious, "that have been processed from an initial total of", len(cuebiq_id_list),"and", len(cuebiq_id_list_remaining), "yet to be processed")
        cuebiq_id_list = cuebiq_id_list_remaining

    print("cuebiq_id_list now has",  len(cuebiq_id_list_remaining), "IDs")
    return cuebiq_id_list, nTotalPrevious

In [ ]:
cuebiq_id_list, nTotalPrevious =EliminatePreviouslyFoundIDs(cuebiq_id_list)
nTotalPrevious

# Query of our dedicated table with pings observed within the Gulf polygon

In [ ]:
# Notes on query speeds for different sizes
## With nIDS=10 it took" 22 seconds
## With nIDS=100 it took" 98 seconds
## With nIDS= 500 it took 102.6 seconds
## With nIDS= 1000 it took 130.6 seconds
## With nIDS= 2000 it took 159.9 seconds
## With nIDS= 10000 it CRASHED!!!

import warnings
warnings.filterwarnings("ignore")

############# Load the PKL
cuebiq_id_list=LoadPkl()

############# Eliminate IDs that have been processed
cuebiq_id_list, nTotalPrevious =EliminatePreviouslyFoundIDs(cuebiq_id_list)

############################################ Numbers that are choices ############################################
# Number of IDs to be drawn at a time and target total
nIDs = 5000                 # of ID's to search for at a time
nTotalIDs = 300000                   # This is the total number it must be greater than the number that is indicated above

############################################ Count numbers ############################################
iRun = 0
nTotal = nTotalPrevious              # This drops from the count the number of IDs that were previously drawn

progress_bar = tqdm(total=nTotalIDs, desc="Processing")
while nTotal < nTotalIDs:
    progress_bar.update(1)
    if iRun > 0:
        del cuebiq_id_list[:nIDs]
    # print("First value in cuebiq_id_list is ", cuebiq_id_list[0])
    iRun = iRun + 1

    #Pull nIDs from convert to a tuple and a data frame
    cuebiq_id_list_short = cuebiq_id_list[:nIDs]  # Extract the first n IDs
    # print("First value in cuebiq_id_list_short is ", cuebiq_id_list_short[0], "length of list is", len(cuebiq_id_list_short))
    cuebiq_id_tuple = tuple(cuebiq_id_list_short)
    # print("iRun=", iRun, "First value in tuple is ", cuebiq_id_tuple[0])

    cuebiq_id_df = pd.DataFrame({'cuebiq_id': cuebiq_id_list_short})
    
    # Main query section
    max_retries = 5
    retry_count = 0
    while retry_count < max_retries:
        try:
            ##########  Main Query #############
            current_time = datetime.now(local_timezone)
            formatted_time = current_time.strftime("%H:%M:%S")
            print(formatted_time, "Before Query. Starting with", cuebiq_id_tuple[0])

            # Create the SQL query to retrieve rows where cuebiq_id is in the first 10 values
            start_time = time.time()
            specific_cuebiq_id_query = f"""
                SELECT *
                FROM dedicated.ScheduledExecution5.DeviceTable
                WHERE cuebiq_id IN {cuebiq_id_tuple}
            """

            specific_cuebiq_id_data = sql_engine.read_sql(specific_cuebiq_id_query)
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            count_unique_cuebiq_ids_1 = specific_cuebiq_id_data['cuebiq_id'].nunique()

            print("With nIDS=", nIDs, "it took" , elapsed_time, "seconds. ", count_unique_cuebiq_ids_1, "IDs were found with", len(specific_cuebiq_id_data), "pings")
            break
        except Exception as e:                
            # Handle the 502 Bad Gateway error
            print(f"An error occurred: {e}")
            retry_count += 1
            time.sleep(10)  # Adjust the delay time as needed
            ##########  End of Main Query #############

    # Remove pings on the islands
    start_time = time.time()
    Gulf_Pings_df = filter_points_outside_polygons(specific_cuebiq_id_data, polygon_wkts)
    # Gulf_Pings_df = is_point_outside_all_islands(specific_cuebiq_id_data)
    Gulf_Pings_df = Gulf_Pings_df.sort_values(by=['cuebiq_id', 'event_timestamp'])
    count_unique_cuebiq_ids_2 = Gulf_Pings_df['cuebiq_id'].nunique()
    elapsed_time = time.time() - start_time
    print("It took", elapsed_time, "seconds to remove island pings. ", count_unique_cuebiq_ids_2, "IDs were found with pings in off the islands, with ", len(Gulf_Pings_df), "pings")

    # Keep only columns with useful information
    columns_to_keep = ['cuebiq_id', 'lat', 'lng', 'event_timestamp']
    Gulf_Pings_df = Gulf_Pings_df[columns_to_keep]

    # Save Gulf Pings to the CSV File
    Gulf_Pings_df.to_csv(Pings_OurTable_Gulf_filename, mode='a', index=False, header=not os.path.exists(Pings_OurTable_Gulf_filename))
    # Save IDs that have already been pulled
    
    # Modify cuebiq_id_df putting the number of pings in the Gulf that are found for that id
    # Create ping_count_df with counts in the Gulf
    ping_count_df = Gulf_Pings_df['cuebiq_id'].value_counts().reset_index()
    ping_count_df.columns = ['cuebiq_id', 'ping_count']

    # Merge ping_count_df with cuebiq_id_df and fill missing ping_count values with 0 and convert everything to integers
    cuebiq_id_df = cuebiq_id_df.merge(ping_count_df, on='cuebiq_id', how='left')
    cuebiq_id_df['ping_count'].fillna(0, inplace=True)
    cuebiq_id_df['ping_count'] = cuebiq_id_df['ping_count'].astype(int)
    cuebiq_id_df['Indicators_Created'] = False
    cuebiq_id_df['Trips_Found'] = 0

    # Save cuebiq_id_df to a csv file
    cuebiq_id_df.to_csv(IDs_Pulled_from_Dedicated_Table_filename, mode='a', index=False, header=not os.path.exists(IDs_Pulled_from_Dedicated_Table_filename))
    
    # Remove the nIDs that were just processed and go back to the top
    nTotal = nTotal + nIDs

    # Print out end of loop
    current_time = datetime.now(local_timezone)
    formatted_time = current_time.strftime("%H:%M:%S")
    print(formatted_time, "There have been a total of", nTotal, "ids processed \n\n")

current_time = datetime.now(local_timezone)
formatted_time = current_time.strftime("%H:%M:%S")
print(formatted_time, "DONE:", nTotal, "IDs have been pulled from the dedicated table")
